In [ ]:
import numpy as np
import os
import random
from random import shuffle
import scipy.misc
import scipy
import time
import skimage.io
from skimage.io import imread
import keras
import keras.callbacks
from keras import optimizers, metrics, regularizers
from keras import backend as K
from keras.models import Sequential, Model, load_model
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization, SeparableConv2D, core
from keras.layers import Conv2D, MaxPooling2D, UpSampling2D, Conv2DTranspose, Cropping2D, concatenate, Input
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from keras.activations import softmax
import matplotlib
import sklearn.metrics
from sklearn.metrics import classification_report, confusion_matrix

class_weight_one = 1
class_weight_two = 1
batch_size = 2
epochs_2_train = 10
validPercent = 0.1
image_folder = '/data/Ro_ImageData/High SN ratio ceramic images/' #Source of images
label_folder = '/data/Ro_ImageData/32xOTSU/' #Otsu images
prediction_folder = 'work/MA499/Predictions_test/' #Folder for predictions

def get_sample(sample,image_folder,label_folder):
    img_int = np.array(imread(image_folder + sample), dtype = 'float32')/255
    lbl_int = np.array(imread(label_folder + sample), dtype = 'float32')/2**16
    lbl_int[:,:,1] = 1-lbl_int[:,:,0]
    lbl_int = lbl_int[:,:,0:2]
    return img_int,lbl_int

def get_sample_test(sample,test_folder):
    img_int = np.array(imread(test_folder + sample), dtype = 'float32')/255
    return img_int

def data_generator(file_name_list,batch_size): #randomly sampled instances from file name list with batch size
    while True:
        batch_filenames = np.random.choice(file_name_list,batch_size)
        batch_input = []
        batch_output = []
        for i_filename in batch_filenames:
            Ai_img, Ai_mask = get_sample(i_filename,image_folder,label_folder)
            batch_input += [Ai_img]
            batch_output += [Ai_mask]
            print(i_filename)
        batch_img = np.expand_dims(np.array(batch_input,dtype='float32'),axis=-1)
        batch_mask = np.array(batch_output,dtype='float32')
        yield (batch_img,batch_mask)

def Weighted_Binary_CrossEntropy(y_true_n,y_pred_n):
    b_ce = K.binary_crossentropy(y_true_n,y_pred_n)
    y_true = K.cast(K.expand_dims(K.argmax(y_true_n,axis=-1),axis=-1),dtype='float32')
    y_pred = K.cast(K.expand_dims(K.argmax(y_pred_n,axis=-1),axis=-1),dtype='float32')
    #Pixel Disparity
    one_weight = class_weight_one
    zero_weight = class_weight_two
    weight_vector = y_true * one_weight + (1.-y_true)*zero_weight
    weighted_b_ce = weight_vector*b_ce
    return K.mean(weighted_b_ce)


sampleList = os.listdir(image_folder)

indexes_valid = random.sample(range(0,len(sampleList)),int(validPercent*float(len(sampleList))))
indexes_train = [x for x in range(0,len(sampleList)) if x not in indexes_valid]
training_list = [x for ind,x in enumerate(sampleList) if ind in indexes_train]
valid_list = [x for ind,x in enumerate(sampleList) if ind in indexes_valid]
#(batch1,batch2) = data_generator(training_list,batch_size)
#print(batch1.shape)
#print(batch2.shape)

'''

test1,test2 = get_sample(training_list[0])
print(np.max(test2))
print(np.min(test2))
matplotlib.image.imsave('test1.png',test1)
matplotlib.image.imsave('test2.png',test2)

'''
num_train_calls = int(float(len(training_list)+1)/float(batch_size))
num_valid_calls = int(float(len(valid_list)+1)/float(batch_size))

#### Network is built ####
network = Sequential()
network.add(Conv2D(10,(3,3),activation = 'relu',kernel_regularizer = regularizers.l2(.01),input_shape = (1024,1024,1),
padding = 'same'))
network.add(keras.layers.BatchNormalization())
network.add(Conv2D(10,(3,3),activation = 'relu',kernel_regularizer = regularizers.l2(.01),padding='same'))
network.add(keras.layers.BatchNormalization())
network.add(Dropout(0.25))
network.add(Conv2D(10,(3,3),activation = 'relu',kernel_regularizer = regularizers.l2(.01),padding='same'))
network.add(keras.layers.BatchNormalization())
network.add(Dropout(0.25))
network.add(Conv2D(10,(3,3),activation = 'relu',kernel_regularizer = regularizers.l2(.01),padding='same'))
network.add(keras.layers.BatchNormalization())
network.add(Dropout(0.25))
network.add(Conv2D(2,(1,1),activation = 'softmax',kernel_regularizer = regularizers.l2(.01),padding='same'))
sgd = optimizers.SGD(lr = 0.0001, decay = 1e-8, momentum = 0.9, nesterov = False)
network.compile(loss = Weighted_Binary_CrossEntropy, optimizer = sgd, metrics = ['accuracy'])
network.summary()
#network.load_weights('weights.h5')
network_training = network.fit_generator(data_generator(training_list,batch_size),
steps_per_epoch = 50, epochs = epochs_2_train, verbose = 1,validation_data = 
data_generator(valid_list,batch_size),validation_steps = num_valid_calls)
#network.save_weights('weights.h5')
TP = 0 #True Positives
TN = 0 #True Negatives
FP = 0 #False Positives
FN = 0 #False Negatives

for filename in valid_list: #Use for new lists
    img,lbl = get_sample(filename,image_folder,label_folder)
    lbl = np.argmax(lbl,axis=-1)
    img = np.expand_dims(img,axis=0)
    img = np.expand_dims(img,axis=-1)
    img_pred = network.predict(img)
    pred = np.argmax(img_pred[0,:,:,:],axis=-1)
    TP += np.sum(pred*lbl,dtype='float32')
    TN += np.sum((1-pred)*(1-lbl),dtype='float32')
    FP += np.sum((1-pred)*lbl,dtype='float32')
    FN += np.sum(pred*(1-lbl),dtype='float32')

ACC = (TP + TN)/(TP + TN + FP + FN) #Accuracy
Recall = (TP)/(TP + FN)
Precision = (TP)/(TP + FP)
MCC = ((TP * TN) - (FP * FN))/np.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))

np.savetxt('ValidationMetrics2.txt',(ACC,Recall,Precision,MCC))
np.savetxt('ValidationLoss2.txt',network_training.history['val_loss'])
np.savetxt('ValidationAcc2.txt',network_training.history['val_acc'])

'''
plt.plot([1,2])
plt.ylabel(network_training.history['val_acc'])
plt.show()
'''

test_list = valid_list

### Generate predictions from trained model ###
for filename in test_list:
    img,lbl = get_sample(filename,image_folder,label_folder)
    lbl = np.argmax(lbl,axis=-1)
    img = np.expand_dims(img,axis=0)
    img = np.expand_dims(img,axis=-1)
    img_pred = network.predict(img)
    pred = np.argmax(img_pred[0,:,:,:],axis=-1)
    matplotlib.image.imsave(prediction_folder+filename,pred)


Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_21 (Conv2D)           (None, 1024, 1024, 10)    100       
_________________________________________________________________
batch_normalization_17 (Batc (None, 1024, 1024, 10)    40        
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 1024, 1024, 10)    910       
_________________________________________________________________
batch_normalization_18 (Batc (None, 1024, 1024, 10)    40        
_________________________________________________________________
dropout_13 (Dropout)         (None, 1024, 1024, 10)    0         
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 1024, 1024, 10)    910       
_________________________________________________________________
batch_normalization_19 (Batc (None, 1024, 1024, 10)   

/opt/conda/lib/python3.7/site-packages/skimage/external/tifffile/tifffile.py:2616: RuntimeWarning: py_decodelzw encountered unexpected end of stream
  strip = decompress(strip)


Tile_r006_c022.tiff
Tile_r047_c024.tiff
Tile_r022_c012.tiff
Tile_r025_c035.tiff
Tile_r035_c003.tiff
Tile_r045_c007.tiff
Tile_r019_c042.tiff
Tile_r016_c044.tiff
Tile_r004_c047.tiff
Tile_r014_c016.tiff
Tile_r014_c030.tiff
Tile_r043_c007.tiff
Tile_r012_c010.tiff
Tile_r043_c007.tiff
Tile_r041_c028.tiff
Tile_r022_c015.tiff
Tile_r000_c019.tiff
Tile_r001_c036.tiff
Tile_r026_c018.tiff
Tile_r019_c013.tiff
Tile_r024_c045.tiff
Tile_r014_c046.tiff
Tile_r044_c016.tiff
Tile_r026_c045.tiff
Tile_r000_c020.tiff
Tile_r000_c001.tiff
Tile_r045_c019.tiff
Tile_r010_c012.tiff
Tile_r010_c007.tiff
Tile_r030_c008.tiff
Tile_r008_c029.tiff
Tile_r009_c027.tiff
Tile_r025_c012.tiff
Tile_r018_c041.tiff
Tile_r040_c033.tiff
Tile_r002_c015.tiff
Tile_r020_c023.tiff
Tile_r039_c015.tiff
Tile_r003_c010.tiff
 1/50 [..............................] - ETA: 1:07 - loss: 0.9587 - accuracy: 0.7207Tile_r011_c006.tiff
Tile_r000_c018.tiff
Tile_r000_c003.tiff
 2/50 [>.............................] - ETA: 36s - loss: 0.9621 - accuracy:

/opt/conda/lib/python3.7/site-packages/skimage/external/tifffile/tifffile.py:2551: UserWarning: unpack: buffer size must be a multiple of element size
  warnings.warn("unpack: %s" % e)


Tile_r046_c023.tiff
36/50 [====================>.........] - ETA: 2s - loss: 0.9103 - accuracy: 0.7620Tile_r011_c002.tiff
Tile_r024_c008.tiff
37/50 [=====================>........] - ETA: 2s - loss: 0.9102 - accuracy: 0.7628Tile_r032_c020.tiff
Tile_r011_c029.tiff
38/50 [=====================>........] - ETA: 2s - loss: 0.9084 - accuracy: 0.7645Tile_r021_c032.tiff
Tile_r005_c020.tiff
39/50 [======================>.......] - ETA: 1s - loss: 0.9060 - accuracy: 0.7662Tile_r038_c008.tiff
Tile_r021_c026.tiff
40/50 [=======================>......] - ETA: 1s - loss: 0.9036 - accuracy: 0.7681Tile_r002_c018.tiff
Tile_r007_c045.tiff
41/50 [=======================>......] - ETA: 1s - loss: 0.9015 - accuracy: 0.7694Tile_r028_c030.tiff
Tile_r038_c020.tiff
42/50 [========================>.....] - ETA: 1s - loss: 0.8994 - accuracy: 0.7712Tile_r016_c039.tiff
Tile_r037_c004.tiff
43/50 [========================>.....] - ETA: 1s - loss: 0.8975 - accuracy: 0.7724Tile_r019_c009.tiff
Tile_r002_c044.tiff
44/5